# Import Data

In [7]:
import json
import ast
import sys
import os
import pandas as pd
import nltk
import re

if not os.getcwd().endswith('UROP_mat2vec'):
    print('Please ensure that you are in the proper working directory')

os.chdir('./UROP_mat2vec')

from utils.process import MaterialsTextProcessor
text_processor = MaterialsTextProcessor()



with open('data/relevant_articles_material_science.json') as json_file:
    relevant = json.load(json_file)

relevant_articles = ast.literal_eval(relevant)
relevant_df = pd.DataFrame({'abstracts' : relevant_articles})

with open('data/irrelevant_articles.json') as json_file:
    irrelevant = json.load(json_file)
    
irrelevant_articles = [i for i in irrelevant if i != '']

for fname in os.listdir('.'):
    if 'irrelevant_articles_' in fname:
        with open('data/' + fname) as json_file:
            irrelevant_articles += ast.literal_eval(json.load(json_file))
            
irrelevant_df = pd.DataFrame({'abstracts' : irrelevant_articles})

Please ensure that you are in the proper working directory


ModuleNotFoundError: No module named 'regex'

# Process Abstracts

1. Remove foreign language
2. Remove common words like "Abstracts"


In [7]:
words = set(nltk.corpus.words.words())

def language_checker(string):
    remove_foreign = " ".join(w for w in nltk.wordpunct_tokenize(string) if w.lower() in words or not w.isalpha())
    # If more than 30% of words are cut, remove article, else keep
    if len(remove_foreign)/len(string) <= 0.5:

        return ""
    else:
        return string
    
def remove_weblinks(string):
    return re.sub('https?://[A-Za-z0-9./]+','', string)
    
relevant_df = relevant_df[relevant_df['abstracts'].apply(language_checker) != ""]
irrelevant_df = irrelevant_df[irrelevant_df['abstracts'].apply(language_checker) != ""]
relevant_df = relevant_df[relevant_df['abstracts'].apply(remove_weblinks) != ""]
irrelevant_df = irrelevant_df[irrelevant_df['abstracts'].apply(remove_weblinks) != ""]
irrelevant_df['abstracts'] = irrelevant_df['abstracts'].apply(lambda x: x.replace("Background", ""))
irrelevant_df['abstracts'] = irrelevant_df['abstracts'].apply(lambda x: x.replace("Abstract", ""))
relevant_df['abstracts'] = relevant_df['abstracts'].apply(lambda x: x.replace("Background", ""))
relevant_df['abstracts'] = relevant_df['abstracts'].apply(lambda x: x.replace("Abstract", ""))
relevant_df['label'] = 1
irrelevant_df['label'] = 0
df = pd.concat([relevant_df, irrelevant_df])

# Train Test split

In [8]:
from sklearn.model_selection import train_test_split

x = df.abstracts
y = df.label

SEED = 2000
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=SEED)


In [9]:
print(y_train[y_train == 1].shape)
print(y_train[y_train == 0].shape)

print(y_test[y_test == 1].shape)
print(y_test[y_test == 0].shape)

(733,)
(3198,)
(302,)
(1384,)


In [28]:
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.neighbors import NearestCentroid
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from time import time
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from xgboost import XGBClassifier

names = ["Logistic Regression", "Linear SVC", "LinearSVC with L1-based feature selection", "Multinomial NB", 
         "Bernoulli NB", "Ridge Classifier", "AdaBoost", "Nearest Centroid", "XGBoost"]
classifiers = [LogisticRegression(),
    LinearSVC(),
    Pipeline([('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False))), ('classification', LinearSVC(penalty="l2"))]),
    MultinomialNB(),
    BernoulliNB(),
    RidgeClassifier(),
    AdaBoostClassifier(),
    NearestCentroid(), 
    XGBClassifier(random_state=1,learning_rate=0.01)]

zipped_clf = zip(names,classifiers)

tvec = TfidfVectorizer()
def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    if len(x_test[y_test == 0]) / (len(x_test)*1.) > 0.5:
        null_accuracy = len(x_test[y_test == 0]) / (len(x_test)*1.)
    else:
        null_accuracy = 1. - (len(x_test[y_test == 0]) / (len(x_test)*1.))
    t0 = time()
    model = pipeline.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    train_test_time = time() - t0
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, labels = np.unique(y_pred))
    return accuracy, f1, train_test_time

def classifier_comparator(vectorizer=tvec, n_features=10000, stop_words=None, ngram_range=(1, 1), classifier=zipped_clf):
    result = []
    vectorizer.set_params(stop_words=stop_words, max_features=n_features, ngram_range=ngram_range)
    for n,c in classifier:
        checker_pipeline = Pipeline([('vectorizer', vectorizer),
                                     ('classifier', c)])
        clf_accuracy, f1_score, tt_time = accuracy_summary(checker_pipeline, x_train, y_train, x_test, y_test)
        result.append((n, clf_accuracy, f1_score, tt_time))
    return result

trigram_result = classifier_comparator(n_features=20000 ,ngram_range=(1,3))

trigram_result

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[('Logistic Regression',
  0.994661921708185,
  0.9848739495798319,
  5.244019031524658),
 ('Linear SVC', 0.998220640569395, 0.9950083194675541, 5.576784372329712),
 ('LinearSVC with L1-based feature selection',
  0.9887307236061684,
  0.9675213675213675,
  4.818987131118774),
 ('SVC', 0.8208778173190985, 0.0, 19.786147832870483),
 ('Multinomial NB', 0.9359430604982206, 0.848314606741573, 3.764889717102051),
 ('Bernoulli NB', 0.6441281138790036, 0.5016611295681063, 3.5783746242523193),
 ('Ridge Classifier',
  0.9976275207591934,
  0.9933554817275748,
  4.219410181045532),
 ('AdaBoost', 0.9976275207591934, 0.9933554817275748, 8.682260274887085),
 ('Nearest Centroid',
  0.9578884934756821,
  0.8945022288261515,
  7.156768560409546),
 ('XGBoost', 0.99644128113879, 0.9899665551839464, 19.569392681121826)]

In [23]:
pipeline = Pipeline([('vectorizer', TfidfVectorizer()), ('classifier', LogisticRegression())])
model = pipeline.fit(x_train, y_train)
y_pred = model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f1

0.9780033840947547

In [18]:
sum(model.predict(relevant_df['abstracts']))
sum(model.predict(irrelevant_df['abstracts']))

0

In [19]:
text_processor = MaterialsTextProcessor()
text_processor.process("LiCoO2 is a battery cathode material.")

(['CoLiO2', 'is', 'a', 'battery', 'cathode', 'material', '.'],
 [('LiCoO2', 'CoLiO2')])

'C:\\Users\\Dell\\OneDrive\\Documents\\School\\Year 4 Sem 1 (SMU)\\UROP\\Logistic Regression'